## ✅ 전제 조건
- Google Colab에서는 기본적으로 GPU 1개만 제공돼서,
"진짜 모델 병렬화"는 안 되지만,
구조 연습은 가능합니다.

- 아래 코드는 PyTorch 공식 예제를 바탕으로,
2개의 GPU가 있다고 가정하고 구성했어요.
GPU가 1개일 때도 실행은 되며, 모델을 두 개의 장치로 나누는 구조 자체를 실습할 수 있습니다

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# 사용할 장치들 확인
device0 = torch.device("cuda:0" if torch.cuda.device_count() > 0 else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.device_count() > 1 else "cpu")

print("사용 가능한 GPU 수:", torch.cuda.device_count())

사용 가능한 GPU 수: 1


In [2]:
# 간단한 2층 모델 정의 - 각 층을 다른 GPU에 올릴 거예요
class ModelParallelNet(nn.Module):
    def __init__(self):
        super(ModelParallelNet, self).__init__()
        # Layer1은 device0에
        self.layer1 = nn.Linear(1024, 2048).to(device0)
        self.relu = nn.ReLU()
        # Layer2는 device1에
        self.layer2 = nn.Linear(2048, 10).to(device1)

    def forward(self, x):
        # 입력은 device0에 있다고 가정
        x = x.to(device0)
        x = self.layer1(x)
        x = self.relu(x)

        # x를 device1로 이동
        x = x.to(device1)
        x = self.layer2(x)
        return x

In [3]:
# 모델 생성
model = ModelParallelNet()

In [4]:
# 옵티마이저는 모든 파라미터를 포함해야 함
optimizer = optim.SGD([
    {'params': model.layer1.parameters()},
    {'params': model.layer2.parameters()}
], lr=0.01)

# 손실 함수
criterion = nn.CrossEntropyLoss()

In [5]:
# 가짜 입력 데이터 생성 (배치 크기 64, 입력 1024차원)
inputs = torch.randn(64, 1024)
labels = torch.randint(0, 10, (64,)).to(device1) # 최종 출력을 기준으로 device1에 label 위치

In [6]:
# 학습 1회
model.train()
optimizer.zero_grad()
outputs = model(inputs)
loss = criterion(outputs, labels)
loss.backward()

optimizer.step()

print('한 번의 학습 완료')

한 번의 학습 완료


| 부분                   | 설명                                  |
| -------------------- | ----------------------------------- |
| `device0`, `device1` | GPU가 2개 있는 경우 각각 `cuda:0`, `cuda:1` |
| `layer1` → `device0` | 첫 번째 레이어는 첫 번째 GPU에 배치              |
| `layer2` → `device1` | 두 번째 레이어는 두 번째 GPU에 배치              |
| `.to(device1)`       | 중간 결과를 다음 GPU로 이동                   |
| `optimizer`          | 두 레이어의 파라미터를 모두 포함해야 함              |
| `labels`             | 출력 레이어가 있는 `device1`에 있어야 함         |


## ✅ 실행 시 주의사항
- 대부분의 Colab에서는 torch.cuda.device_count()가 1입니다.

- GPU가 하나면 device1 = cuda:0과 동일하게 되고, 코드가 문제 없이 실행돼요.

- 구조 자체를 이해하고 연습하는 데 목적이 있다면 Colab에서도 충분히 가능합니다.

## 📦 출력 예시 (GPU 1개인 경우)
```
사용 가능한 GPU 수: 1
한 번의 학습 완료
```